In [84]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

## 1. Load Churn Modelling Dataset

In [85]:
RANDOM_STATE = 42

In [ ]:
# get the root project's path
PROJECT_DIRPATH = Path.cwd().parent
PROJECT_DIRPATH

In [87]:
# get and create (if not exist) the models' path
MODELS_DIRPATH = PROJECT_DIRPATH / 'models'
MODELS_DIRPATH.mkdir(exist_ok=True)

In [ ]:
# construct dataset filepath
DATASET_FILENAME = "churn_modelling.csv"
DATASET_FILEPATH = Path.joinpath(PROJECT_DIRPATH, "data", "churn_modelling.csv")
DATASET_FILEPATH

In [ ]:
data = pd.read_csv(DATASET_FILEPATH)
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

## 2. Data Preprocessing

### 2.1 Drop Irrelevant Columns

In [ ]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

### 2.2 Encode Categorical Values

In [ ]:
#  Encode 'Gender' values with LabelEncoder
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
# Encode 'Geography' values with OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded.toarray()

In [ ]:
geo_encoded_columns = onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=geo_encoded_columns)
geo_encoded_df

In [ ]:
# Combine one-hot encoded 'Geography' columns with the original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

### 2.3 Save the Encoders for Future Uses

In [97]:
with open(MODELS_DIRPATH / 'label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open(MODELS_DIRPATH / 'onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

### 2.4 Split the Data

In [ ]:
data.head()

In [99]:
# Divide the dataset into independent and dependent features
X = data.drop('Exited', axis=1)
y = data['Exited']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)


### 2.5 Scale Features

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train, X_test

### 2.6 Save the Scalers for Future Uses

In [101]:
with open(MODELS_DIRPATH / 'scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)